<a href="https://colab.research.google.com/github/eduseiti/ia368v_dd_final/blob/master/colbertx_train_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys

In [ ]:
IN_COLAB='google.colab' in sys.modules

In [ ]:
if IN_COLAB:
    from google.colab import drive

    WORKING_FOLDER="/content/drive/MyDrive/unicamp/ia368v_dd/trabalho_final_UNICAMP-IR"

    drive.mount('/content/drive', force_remount=True)

    os.chdir(WORKING_FOLDER)

    !pip install transformers -q
    !pip install mlflow -q
    !pip install ujson -q
    !pip install faiss-gpu -q
    !pip install SentencePiece -q

    if not os.path.exists("ColBERT-X"):
        !git clone https://github.com/hltcoe/ColBERT-X.git

    PYTHON="python3"
else:
    WORKING_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/trabalho_final/"

    PỲTHON="python3.8"

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 90.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62

In [ ]:
CLUEWEB22_FOLDER="clueweb22-pt_10M_sample"

CORPUS_PART_INDEX_FORMAT="clueweb22-pt_10M_sample_part_{:02}_index"

QUERIES_FILE="train_queries.tsv"

In [ ]:
INDEX_PARTS_PARAM=[
    '0..3',
    '3..6',
    '6..9',
    '9..10'
]

In [ ]:
os.chdir("ColBERT-X")

## Execute the retrieval over the 5 corpus parts, going through the colbertx index in chunks of 3

In [ ]:
for corpus_part in range(5):

	corpus_part_index = CORPUS_PART_INDEX_FORMAT.format(corpus_part)

	print("\n\nSearching corpus part {}: colbertx index: {}".format(corpus_part, corpus_part_index))

	for colbertx_indexes in INDEX_PARTS_PARAM:

		print(">> Index parts: {}".format(colbertx_indexes))

		!python3 -m xlmr_colbert.retrieve --similarity l2 --dim 48 --query_maxlen 32 \
			--index_root ../{CLUEWEB22_FOLDER} --index_name {corpus_part_index} --faiss_name ivfpq.262144.faiss --part-range {colbertx_indexes} --log_scores \
			--retrieve_only --queries ../{QUERIES_FILE} \
			--amp --checkpoint ../colbertx_experiments/mMSMARCO-pt_048_dim/train.py/fine_tune_025/checkpoints/colbert-20000.dnn

Streaming output truncated to the last 5000 lines.
1362 Quais são as características dos principais locais turísticos? 1708 1708 28.07533836364746 227712 131.3511457674134 ms
1363 Como a teoria dos polos de Perroux foi aplicada no planejamento do território brasileiro e quais foram seus efeitos? 4053 4053 23.870086669921875 261561 131.36979386953314 ms
1364 Qual a importância de um planejamento financeiro a médio e longo prazo? 1712 1712 26.99537467956543 303139 131.37152186243526 ms
1365 Como calcular a reserva de emergência e como ela pode ajudar em momentos de crise financeira? 3302 3302 26.54279327392578 443090 131.38245279583018 ms
1366 Qual é a importância de completar outras quests antes da missão "Jornada a Meridian"? 3620 3620 24.920820236206055 530710 131.38896580063346 ms
1367 Quais são as missões necessárias para garantir a sobrevivência da Capitã Dunn na batalha final? 3349 3349 24.31256866455078 22507 131.4068835729744 ms
1368 O que fazer em caso de acidente de trânsito? 

In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
corpus_part = 4
corpus_part_index = CORPUS_PART_INDEX_FORMAT.format(corpus_part)
colbertx_indexes = '9..10'


print("\n\nSearching corpus part {}: colbertx index: {}".format(corpus_part, corpus_part_index))

print(">> Index parts: {}".format(colbertx_indexes))

!python3 -m xlmr_colbert.retrieve --similarity l2 --dim 48 --query_maxlen 32 \
    --index_root ../{CLUEWEB22_FOLDER} --index_name {corpus_part_index} --faiss_name ivfpq.262144.faiss --part-range {colbertx_indexes} --log_scores \
    --retrieve_only --queries ../{QUERIES_FILE} \
    --amp --checkpoint ../colbertx_experiments/mMSMARCO-pt_048_dim/train.py/fine_tune_025/checkpoints/colbert-20000.dnn



Searching corpus part 4: colbertx index: clueweb22-pt_10M_sample_part_04_index
>> Index parts: 9..10
2023-06-26 21:28:06.208713: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-26 21:28:06.262094: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-26 21:28:07.176982: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[Jun 26, 21:28:21] #> Creating directory /content/drive/MyDrive/unicamp/ia368v_dd/trabalho_final_UNICAMP-IR/ColBERT-X/experiments/dirty/retrieve.